In [1]:
import numpy as np
import os
from soma import aims, aimsalgo
import pandas as pd

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
directory = '/volatile/jl277509/data/UkBioBank/crops/2mm/CINGULATE/mask/'
side = 'R'

In [3]:
crops_dirs = directory+side+'crops/'
skel_subjects = pd.read_csv(directory+side+'skeleton_subject.csv')

In [18]:
distbottom_list = []
fm = aims.FastMarching()
for i, subject in enumerate(skel_subjects.Subject):
    vol = aims.read(crops_dirs+subject+'_cropped_skeleton.nii.gz')
    print(np.unique(vol.np))
    vol[vol.np > 30] = 100
    print(np.unique(vol.np))
    print(np.sum(vol.np>0))
    # carte de distance par FastMarching
    # propagation dans le label 100 (squelette) à partir de la graine 30 (bottom), sans graine extérieure (-1 n'existe pas dans l'image)
    dist = fm.doit(vol, 100, 30, -1)
    # convertir en np, modifier le type, puis reconvertir en vol int16
    arr_to_write = dist.np
    print(np.unique(arr_to_write))
    print(np.sum(arr_to_write<10000))
    outside = np.max(arr_to_write)
    inf_dist=2**15-1
    arr_to_write[arr_to_write==outside]=inf_dist
    arr_to_write = arr_to_write.astype(np.int16)
    arr_to_write = np.array(arr_to_write, order='F')
    print(np.sum(arr_to_write<10000))
    vol_to_write = aims.Volume(arr_to_write)
    #aims.write(vol_to_write, directory+side+f'distbottom/{subject}_cropped_distbottom.nii.gz')
    distbottom_list.append(arr_to_write)
    break

[  0  30  35  60 100 120]
[  0  30 100]
376
[0.00000000e+00 1.41421354e+00 1.93185163e+00 2.00000000e+00
 3.09065795e+00 3.41421366e+00 3.80132914e+00 4.00000000e+00
 4.88446140e+00 5.09065819e+00 5.41421366e+00 5.80132914e+00
 6.00000000e+00 6.39801025e+00 6.88446140e+00 7.09065628e+00
 7.09065819e+00 7.21554279e+00 7.41421366e+00 8.00000000e+00
 8.03437710e+00 8.65736294e+00 9.09065533e+00 9.09065628e+00
 9.21554279e+00 9.41421318e+00 9.93185139e+00 1.00000000e+01
 1.00343771e+01 1.02715340e+01 1.05048685e+01 1.06297560e+01
 1.06573591e+01 1.06573629e+01 1.10633669e+01 1.10906563e+01
 1.14142132e+01 1.19318514e+01 1.20000000e+01 1.20343771e+01
 1.26297560e+01 1.26573591e+01 1.28435097e+01 1.34142132e+01
 1.40000000e+01 1.46573591e+01 1.54142132e+01 1.60000000e+01
 1.66573601e+01 1.80000000e+01 2.00000000e+01 3.40282347e+38]
297
297


In [ ]:
arr1 = vol.np == 30
arr2 = dist.np == 0
np.sum(arr1!=arr2)

In [13]:
print(np.sum(vol.np>0))
print(np.sum(arr_to_write<10000))

376
297


In [43]:
arr_distbottom_all = np.stack(distbottom_list)

In [44]:
arr_distbottom_all.shape

(21052, 16, 37, 37, 1)

In [54]:
np.random.randint(0,1)

0

In [45]:
np.save(directory+side+'distbottom.npy', arr_distbottom_all)

In [32]:
a = np.load('/volatile/jl277509/data/UkBioBank/crops/2mm/CINGULATE/mask/Rskeleton.npy')

In [33]:
a.shape

(21052, 16, 37, 37, 1)

In [29]:
b = np.load('/neurospin/dico/data/deep_folding/current/datasets/ACCpatterns/crops/2mm/CINGULATE/mask/Rskeleton.npy')

In [30]:
b.shape

(341, 16, 37, 37, 1)